## 任务4 ：轻量级MCP工具开发——代码级实践

### 技术细节解释

这个高精度除法 MCP 服务端基于 **FastAPI** 构建，并遵循了 MCP 协议的一些关键约定。

#### FastAPI 框架

* **`FastAPI`**: 一个现代、快速（高性能）的 Python Web 框架，用于构建 API。它基于 Starlette (高性能 ASGI 框架) 和 Pydantic (数据验证和设置管理)。
* **`app = FastAPI(...)`**: 初始化 FastAPI 应用，并设置了 `title`、`description` 和 `version`。这些信息在访问 `/` 根路径时返回，有助于 API 的自我描述。

#### 高精度计算 (`decimal` 模块)

* **`from decimal import Decimal, getcontext`**: Python 内置的 `decimal` 模块提供了任意精度的十进制浮点数算术。标准的 `float` 类型是基于 IEEE 754 双精度浮点数，在表示某些小数时可能存在精度问题（例如 `0.1 + 0.2` 不等于 `0.3`）。
* **`getcontext().prec = 50`**: 这行代码设置了 `decimal` 模块的全局精度。`prec` 参数定义了计算结果的有效数字位数。这里设置为 50 位，意味着在进行除法运算时，结果将保留至少 50 位有效数字，从而实现“高精度”。
* **`num1: Decimal`, `num2: Decimal`, `result: Decimal`**: 在 Pydantic 模型中，将数字类型定义为 `Decimal`。FastAPI 会自动处理请求体中浮点数到 `Decimal` 对象的转换，并在响应中将 `Decimal` 转换回 JSON 可识别的数字或字符串。

#### API 端点 (`@app.post`, `@app.get`)

* **`@app.post("/divide", ...)`**: 定义了一个 HTTP POST 请求的 `/divide` 端点。POST 请求通常用于发送数据以创建或执行操作。
    * `response_model=Result`: 指定了该端点的响应将符合 `Result` Pydantic 模型的结构。
    * `summary`, `description`, `operation_id`: 这些是 OpenAPI (Swagger UI) 规范的元数据，用于自动生成 API 文档，使得工具的功能更易于理解。
* **`async def divide_numbers(numbers: Numbers) -> Result:`**: 这是处理 `/divide` 请求的异步函数。
    * `numbers: Numbers`: FastAPI 会自动解析请求体中的 JSON 数据，并将其验证为 `Numbers` Pydantic 模型的实例。
    * **除零处理**: `if numbers.num2 == 0: raise HTTPException(...)` 这是一个关键的错误处理。如果除数为零，服务器会返回 `HTTP 400 Bad Request` 状态码和相应的错误详情，而不是抛出运行时错误。
    * `numbers.num1 / numbers.num2`: 执行高精度除法。
* **`@app.get("/capabilities", ...)`**: 这是一个符合 MCP 协议标准的端点。AI Agent (或 MCP Client) 会访问这个端点来发现服务器提供了哪些工具以及如何使用它们（包括参数、描述和安全要求）。
    * 这里返回的 JSON 结构描述了 `divide` 工具的名称、描述、所需参数 (`num1`, `num2` 都是 `number` 类型且为必填) 以及安全要求（本例中无安全要求，所以 `security` 字段被省略或为空）。
* **`@app.get("/")` 和 `@app.post("/")`**: 根路径端点，提供 API 的基本信息和处理根路径的 POST 请求。这有助于 API 的可发现性和兼容性。

#### 数据验证和序列化 (Pydantic)

* **`BaseModel`**: `Numbers` 和 `Result` 类继承自 Pydantic 的 `BaseModel`。Pydantic 提供了强大的数据验证和序列化功能。
* 当请求到达 `/divide` 端点时，FastAPI 会自动将传入的 JSON 数据验证为 `Numbers` 模型的实例。如果数据格式不正确（例如，`num1` 不是数字），FastAPI 会自动返回 `HTTP 422 Unprocessable Entity` 错误。
* 在返回响应时，Pydantic 会将 `Result` 实例自动序列化为 JSON 格式。

#### 无认证机制

* 与之前的示例不同，这个 `server.py` 文件中**移除了所有与 API 密钥认证相关的代码**（例如 `APIKeyHeader`、`verify_api_key` 函数以及在端点中对 `Depends(verify_api_key)` 的引用）。这意味着任何人都可以访问这个除法服务，无需提供密钥。在生产环境中，这通常不建议，除非服务本身就是公开的。

#### Uvicorn 服务器

* **`uvicorn.run(app, host="127.0.0.1", port=8000)`**: 这行代码在 `if __name__ == "__main__":` 块中，表示当直接运行 `server.py` 文件时，Uvicorn 会启动一个 ASGI 服务器，监听 `127.0.0.1:8000`，并运行我们的 FastAPI 应用。